In [20]:
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
from IPython.core.debugger import Tracer

In [21]:
#Assign random weights
weights = {'Problem Solving': 80, 'Design': 80, 'CS Skills': 50, 'Test Enumeration': 40, 'Communication': 30}

In [22]:
def calculateScore(df):
    Score = []
    for index, row in df.iterrows():
        numerator = 0; denominator = 0;
        for key, value in weights.items():
            if(row[key] != -1):
                numerator += row[key]*value
                denominator += value
        Score.append(numerator/denominator if numerator > 0 else -1)
    print(len(Score))
    return Score
     

In [23]:
def calculateFinalCall(df):
    FinalCall = []
    for x in range (0, 1000):
        if(x % 4 == 0):
            count = 0
            for y in range (0,4):
                count += df.loc[x + y]['Interviewer Call']
            #print(count)
            finalDecision = 1 if count >= 3 else 0
            for z in range (0,4):
                FinalCall.append(finalDecision)
    print(len(FinalCall))
    return FinalCall
        

In [24]:
def computeDecision(score):
    percentile70 = np.percentile(score, 70)
    percentile50 = np.percentile(score, 50)
    print(percentile70)
    if(score >= percentile70):
         x = 1
    elif(score < percentile70 and score >= percentile50):
        x = np.random.randint(2)
    else:
        x = 0
    return x

In [25]:
def script():
    wb = Workbook()

    # grab the active worksheet
    ws = wb.active

    # initializeHeaders(ws)
    
    #Initialize metrics
    df = pd.DataFrame(np.random.randint(low=0, high=12, size=(1000, 5)), columns=['Problem Solving', 'Design', 'CS Skills' , 'Test Enumeration' , 'Communication'])
    
    # Make some fields as -1. Now scores lie within 2 to 9 and empty values are -1
    df = df.applymap(lambda x : (-1 if x in (0, 1, 10, 11) else x))
    
    #Populate intervieweeId
    intervieweeId = []
    for x in range (1, 1001):
        intervieweeId.append(np.ceil(x/4))
    df['intervieweeId'] = intervieweeId
    
    #Populate interviewerId
    df['interviewerId'] = np.random.randint(1, 101, df.shape[0])
    
    #Populate score based on weighted average 
    df['Score'] = calculateScore(df)
    
    
    df['Interviewer Call'] = df.Score.apply(fx)
    
    #Populate final call
    df['Final Call'] = calculateFinalCall(df)
    
    #TODO : Make blow comment work, to check actual how many candidates are hired
    #print(df.groupby('intervieweeId')['Final Call'].filter(lambda x : x['Final Call'] == 1))
    print(df['Final Call'].value_counts())
    
    print(df.head())
    
    for r in dataframe_to_rows(df, index=True, header=True):
        ws.append(r)

    # Save the file
    wb.save("sample.xlsx")